# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [ ]:


from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time



### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [ ]:
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
# 模擬使用者操作行為，選擇/點擊
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

time.sleep(2)

In [ ]:


# 取得資料，丟到 BeautifulSoup 解析
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

th = table.find_all('th')
columns = [tag.text for tag in th]

d = {}
td = table.find_all('td')

for tag in td:
    if tag.attrs and tag.get('style'):
        d.setdefault(tag.text, {})
    elif not tag.attrs and tag.text.strip():
        if '/' in tag.text:
            k = tag.text
        else:
            d[list(d.keys())[-1]].setdefault(k, tag.text)

df = pd.DataFrame(d)
df2 = df[['SO2']].iloc[:8].reset_index()
df2.columns = columns[2:4]
df2



In [ ]:


df = pd.DataFrame(d)
df2 = df[['SO2', 'CO']].iloc[:8].reset_index()
old_columns = list(df2.columns)
new_columns = columns[2:3] + \
              list(map(lambda x: str(x) + '-' + ''.join(columns[3:4]), old_columns[1:]))
df2.columns = new_columns
df2



In [ ]:
browser.quit();